In [2]:
!pip install selenium-stealth


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 임산부

In [ ]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_pregnant_woman.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = (
        "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll"
        "?cityDoArea=ALL&siGunGuArea=ALL&sidocode=ALL&svccd=ALL"
        "&tccd=ALL&meancd=ALL&chktype1=&startCount=0&sortOrder=DESC"
        "&realQuery=%28%EC%9E%84%EC%82%B0%EB%B6%80%29"
        "&sort=RANK&query=%EC%9E%84%EC%82%B0%EB%B6%80&showView=view22"
    )
    driver.get(base_url)

    records = []
    try:
        for page in range(1, 31):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
[페이지 15] 수집 중…
[페이지 16] 수집 중…
[페이지 17] 수집 중…
[페이지 18] 수집 중…
[페이지 19] 수집 중…
[페이지 20] 수집 중…
[페이지 21] 수집 중…
[페이지 22] 수집 중…
[페이지 23] 수집 중…
[페이지 24] 수집 중…
[페이지 25] 수집 중…
[페이지 26] 수집 중…
[페이지 27] 수집 중…
[페이지 28] 수집 중…
[페이지 29] 수집 중…
[페이지 30] 수집 중…
완료: 총 704건 → 'gov_services_pregnant_woman.xlsx'


In [34]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_pregnant_woman.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_pregnant_woman.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_pregnant_woman.json"


cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 컬럼 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
for col in cols:
    df[col] = ""
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns)).to_csv(
        OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig'
    )
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) Selenium 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1, b=3):
    time.sleep(random.uniform(a, b))

def safe_click(el):
    try:
        el.click()
    except Exception:
        driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수 정의
# ——————————————
def scrape_panel1(idx):
    try:
        el = wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner'))
        )
        mapping = {
            'term': '신청기간',
            'call': '전화문의',
            'method': '신청방법',
            'reception': '접수기관',
            'support': '지원형태',
            'legalInfo': '사업근거'
        }
        for cls, col in mapping.items():
            try:
                df.at[idx, col] = el.find_element(
                    By.CSS_SELECTOR, f'li.{cls} span'
                ).text.strip()
            except NoSuchElementException:
                continue
    except TimeoutException:
        pass

def scrape_tab(idx, panel):
    btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[data-tab='{panel}']"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    human_delay(0.5, 1)
    safe_click(btn)
    human_delay(0.5, 1)
    if panel in ['panel2', 'panel3']:
        sel = f"#{panel} .detail-desc, #{panel} .service-detail-inner"
        key = '지원대상' if panel == 'panel2' else '지원내용'
        try:
            df.at[idx, key] = wait.until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, sel))
            ).text.strip()
        except TimeoutException:
            df.at[idx, key] = ''
    elif panel == 'panel4':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel4 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '신청기간' in title:
                    df.at[idx, '신청기간'] = text
                elif '신청방법' in title:
                    df.at[idx, '신청방법'] = text
            except NoSuchElementException:
                continue
    elif panel == 'panel5':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel5 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '접수기관' in title:
                    df.at[idx, '접수기관'] = text
                elif '문의처' in title:
                    df.at[idx, '문의처'] = text
            except NoSuchElementException:
                continue

# ——————————————
# 4) 메인 루프: 모든 행 저장 및 체크포인트 기록
# ——————————————
for idx, row in df.iterrows():
    url = row.get('detail_url')
    if not url:
        continue
    # 한번이라도 처리된 URL 스킵
    if url in checkpoint:
        print(f"[{idx+1}] 스킵됨: {url}")
        continue
    driver.get(url)
    human_delay(1, 2)
    # panel1 스크래핑
    scrape_panel1(idx)
    # 패널 순서대로 스크래핑
    for panel in ['panel3', 'panel2', 'panel4', 'panel5']:
        scrape_tab(idx, panel)
    # 누락 항목 기본값 처리
    missing = [c for c in cols if not df.at[idx, c].strip()]
    for c in missing:
        df.at[idx, c] = ''  # 빈 문자열 그대로 유지
    # CSV 저장
    record = row.to_dict()
    for c in cols:
        record[c] = df.at[idx, c]
    pd.DataFrame([record]).to_csv(
        OUTPUT_CSV_PATH, mode='a', index=False, header=False, encoding='utf-8-sig'
    )
    # 체크포인트 저장 (누락 여부 관계 없이)
    checkpoint[url] = True
    with open(CHECKPOINT_PATH, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)
    print(f"[{idx+1}/{len(df)}] 저장완료: {url}, 누락={missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')




[1/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/628000000735?administOrgCd=ALL, 누락=[]
[2/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/135200000005?administOrgCd=ALL, 누락=[]
[3/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/135200000114?administOrgCd=ALL, 누락=[]
[4/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/567000000111?administOrgCd=ALL, 누락=[]
[5/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/611000000160?administOrgCd=ALL, 누락=[]
[6/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/404000000104?administOrgCd=ALL, 누락=[]
[7/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/414000000117?administOrgCd=ALL, 누락=[]
[8/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/B55265700002?administOrgCd=ALL, 누락=[]
[9/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/567000000104?administOrgCd=ALL, 누락=[]
[10/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/569000000344?administOrgCd=ALL, 누락=[]
[11/704] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/64

### 여성 정책

In [12]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_woman.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%28%EC%9E%84%EC%82%B0%EB%B6%80%29%7C%28%EC%9E%84%EC%82%B0%EB%B6%80%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%EC%97%AC%EC%84%B1"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 15):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
완료: 총 328건 → 'gov_services_woman.xlsx'


In [33]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_woman.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_woman.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_woman.json"


cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 컬럼 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
for col in cols:
    df[col] = ""
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns)).to_csv(
        OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig'
    )
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) Selenium 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1, b=3):
    time.sleep(random.uniform(a, b))

def safe_click(el):
    try:
        el.click()
    except Exception:
        driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수 정의
# ——————————————
def scrape_panel1(idx):
    try:
        el = wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner'))
        )
        mapping = {
            'term': '신청기간',
            'call': '전화문의',
            'method': '신청방법',
            'reception': '접수기관',
            'support': '지원형태',
            'legalInfo': '사업근거'
        }
        for cls, col in mapping.items():
            try:
                df.at[idx, col] = el.find_element(
                    By.CSS_SELECTOR, f'li.{cls} span'
                ).text.strip()
            except NoSuchElementException:
                continue
    except TimeoutException:
        pass

def scrape_tab(idx, panel):
    btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[data-tab='{panel}']"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    human_delay(0.5, 1)
    safe_click(btn)
    human_delay(0.5, 1)
    if panel in ['panel2', 'panel3']:
        sel = f"#{panel} .detail-desc, #{panel} .service-detail-inner"
        key = '지원대상' if panel == 'panel2' else '지원내용'
        try:
            df.at[idx, key] = wait.until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, sel))
            ).text.strip()
        except TimeoutException:
            df.at[idx, key] = ''
    elif panel == 'panel4':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel4 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '신청기간' in title:
                    df.at[idx, '신청기간'] = text
                elif '신청방법' in title:
                    df.at[idx, '신청방법'] = text
            except NoSuchElementException:
                continue
    elif panel == 'panel5':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel5 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '접수기관' in title:
                    df.at[idx, '접수기관'] = text
                elif '문의처' in title:
                    df.at[idx, '문의처'] = text
            except NoSuchElementException:
                continue

# ——————————————
# 4) 메인 루프: 모든 행 저장 및 체크포인트 기록
# ——————————————
for idx, row in df.iterrows():
    url = row.get('detail_url')
    if not url:
        continue
    # 한번이라도 처리된 URL 스킵
    if url in checkpoint:
        print(f"[{idx+1}] 스킵됨: {url}")
        continue
    driver.get(url)
    human_delay(1, 2)
    # panel1 스크래핑
    scrape_panel1(idx)
    # 패널 순서대로 스크래핑
    for panel in ['panel3', 'panel2', 'panel4', 'panel5']:
        scrape_tab(idx, panel)
    # 누락 항목 기본값 처리
    missing = [c for c in cols if not df.at[idx, c].strip()]
    for c in missing:
        df.at[idx, c] = ''  # 빈 문자열 그대로 유지
    # CSV 저장
    record = row.to_dict()
    for c in cols:
        record[c] = df.at[idx, c]
    pd.DataFrame([record]).to_csv(
        OUTPUT_CSV_PATH, mode='a', index=False, header=False, encoding='utf-8-sig'
    )
    # 체크포인트 저장 (누락 여부 관계 없이)
    checkpoint[url] = True
    with open(CHECKPOINT_PATH, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)
    print(f"[{idx+1}/{len(df)}] 저장완료: {url}, 누락={missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')



[1/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/999000000050?administOrgCd=ALL, 누락=[]
[2/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/WII000000860?administOrgCd=ALL, 누락=[]
[3/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/GMW000000120?administOrgCd=ALL, 누락=[]
[4/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/641000000180?administOrgCd=ALL, 누락=[]
[5/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/646000000145?administOrgCd=ALL, 누락=[]
[6/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/GMW000000260?administOrgCd=ALL, 누락=[]
[7/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/GEF000000070?administOrgCd=ALL, 누락=[]
[8/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/648000001116?administOrgCd=ALL, 누락=[]
[9/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/142100000009?administOrgCd=ALL, 누락=[]
[10/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/PTR000052170?administOrgCd=ALL, 누락=[]
[11/328] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/14

### 한부모 

In [20]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_single_parent.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%281%EC%9D%B8+%EA%B0%80%EA%B5%AC%29%7C%28system%3Anumber%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%ED%95%9C%EB%B6%80%EB%AA%A8"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 63):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
[페이지 15] 수집 중…
[페이지 16] 수집 중…
[페이지 17] 수집 중…
[페이지 18] 수집 중…
[페이지 19] 수집 중…
[페이지 20] 수집 중…
[페이지 21] 수집 중…
[페이지 22] 수집 중…
[페이지 23] 수집 중…
[페이지 24] 수집 중…
[페이지 25] 수집 중…
[페이지 26] 수집 중…
[페이지 27] 수집 중…
[페이지 28] 수집 중…
[페이지 29] 수집 중…
[페이지 30] 수집 중…
[페이지 31] 수집 중…
[페이지 32] 수집 중…
[페이지 33] 수집 중…
[페이지 34] 수집 중…
[페이지 35] 수집 중…
[페이지 36] 수집 중…
[페이지 37] 수집 중…
[페이지 38] 수집 중…
[페이지 39] 수집 중…
[페이지 40] 수집 중…
[페이지 41] 수집 중…
[페이지 42] 수집 중…
[페이지 43] 수집 중…
[페이지 44] 수집 중…
[페이지 45] 수집 중…
[페이지 46] 수집 중…
[페이지 47] 수집 중…
[페이지 48] 수집 중…
[페이지 49] 수집 중…
[페이지 50] 수집 중…
[페이지 51] 수집 중…
[페이지 52] 수집 중…
[페이지 53] 수집 중…
[페이지 54] 수집 중…
[페이지 55] 수집 중…
[페이지 56] 수집 중…
[페이지 57] 수집 중…
[페이지 58] 수집 중…
[페이지 59] 수집 중…
[페이지 60] 수집 중…
[페이지 61] 수집 중…
[페이지 62] 수집 중…
완료: 총 1476건 → 'gov_services_single_parent.xlsx'


In [31]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_single_parent.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_single_parent.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_single_parent.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 컬럼 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
for col in cols:
    df[col] = ""
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns)).to_csv(
        OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig'
    )
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) Selenium 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1, b=3):
    time.sleep(random.uniform(a, b))

def safe_click(el):
    try:
        el.click()
    except Exception:
        driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수 정의
# ——————————————
def scrape_panel1(idx):
    try:
        el = wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner'))
        )
        mapping = {
            'term': '신청기간',
            'call': '전화문의',
            'method': '신청방법',
            'reception': '접수기관',
            'support': '지원형태',
            'legalInfo': '사업근거'
        }
        for cls, col in mapping.items():
            try:
                df.at[idx, col] = el.find_element(
                    By.CSS_SELECTOR, f'li.{cls} span'
                ).text.strip()
            except NoSuchElementException:
                continue
    except TimeoutException:
        pass

def scrape_tab(idx, panel):
    btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[data-tab='{panel}']"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    human_delay(0.5, 1)
    safe_click(btn)
    human_delay(0.5, 1)
    if panel in ['panel2', 'panel3']:
        sel = f"#{panel} .detail-desc, #{panel} .service-detail-inner"
        key = '지원대상' if panel == 'panel2' else '지원내용'
        try:
            df.at[idx, key] = wait.until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, sel))
            ).text.strip()
        except TimeoutException:
            df.at[idx, key] = ''
    elif panel == 'panel4':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel4 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '신청기간' in title:
                    df.at[idx, '신청기간'] = text
                elif '신청방법' in title:
                    df.at[idx, '신청방법'] = text
            except NoSuchElementException:
                continue
    elif panel == 'panel5':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel5 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '접수기관' in title:
                    df.at[idx, '접수기관'] = text
                elif '문의처' in title:
                    df.at[idx, '문의처'] = text
            except NoSuchElementException:
                continue

# ——————————————
# 4) 메인 루프: 모든 행 저장 및 체크포인트 기록
# ——————————————
for idx, row in df.iterrows():
    url = row.get('detail_url')
    if not url:
        continue
    # 한번이라도 처리된 URL 스킵
    if url in checkpoint:
        print(f"[{idx+1}] 스킵됨: {url}")
        continue
    driver.get(url)
    human_delay(1, 2)
    # panel1 스크래핑
    scrape_panel1(idx)
    # 패널 순서대로 스크래핑
    for panel in ['panel3', 'panel2', 'panel4', 'panel5']:
        scrape_tab(idx, panel)
    # 누락 항목 기본값 처리
    missing = [c for c in cols if not df.at[idx, c].strip()]
    for c in missing:
        df.at[idx, c] = ''  # 빈 문자열 그대로 유지
    # CSV 저장
    record = row.to_dict()
    for c in cols:
        record[c] = df.at[idx, c]
    pd.DataFrame([record]).to_csv(
        OUTPUT_CSV_PATH, mode='a', index=False, header=False, encoding='utf-8-sig'
    )
    # 체크포인트 저장 (누락 여부 관계 없이)
    checkpoint[url] = True
    with open(CHECKPOINT_PATH, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)
    print(f"[{idx+1}/{len(df)}] 저장완료: {url}, 누락={missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')


[1] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/SSI000000020?administOrgCd=ALL
[2] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/138300000017?administOrgCd=ALL
[3] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/WII000000820?administOrgCd=ALL
[4] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/642000000700?administOrgCd=ALL
[5] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/569000000365?administOrgCd=ALL
[6] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/138300000040?administOrgCd=ALL
[7] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/611000000116?administOrgCd=ALL
[8] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/648000001077?administOrgCd=ALL
[9] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/630000000125?administOrgCd=ALL
[10] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/641000000155?administOrgCd=ALL
[11] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/642000000035?administOrgCd=ALL
[12] 스킵됨: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/WII000001550?administOrgCd=ALL
[13] 스킵됨: htt

### 미혼모

In [22]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_single_mom.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%28%ED%95%9C%EB%B6%80%EB%AA%A8%29%7C%28%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%EB%AF%B8%ED%98%BC%EB%AA%A8#none"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 2):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
완료: 총 22건 → 'gov_services_single_mom.xlsx'


In [32]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_single_mom.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_single_mom.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_single_mom.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 컬럼 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
for col in cols:
    df[col] = ""
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns)).to_csv(
        OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig'
    )
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) Selenium 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1, b=3):
    time.sleep(random.uniform(a, b))

def safe_click(el):
    try:
        el.click()
    except Exception:
        driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수 정의
# ——————————————
def scrape_panel1(idx):
    try:
        el = wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner'))
        )
        mapping = {
            'term': '신청기간',
            'call': '전화문의',
            'method': '신청방법',
            'reception': '접수기관',
            'support': '지원형태',
            'legalInfo': '사업근거'
        }
        for cls, col in mapping.items():
            try:
                df.at[idx, col] = el.find_element(
                    By.CSS_SELECTOR, f'li.{cls} span'
                ).text.strip()
            except NoSuchElementException:
                continue
    except TimeoutException:
        pass

def scrape_tab(idx, panel):
    btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[data-tab='{panel}']"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    human_delay(0.5, 1)
    safe_click(btn)
    human_delay(0.5, 1)
    if panel in ['panel2', 'panel3']:
        sel = f"#{panel} .detail-desc, #{panel} .service-detail-inner"
        key = '지원대상' if panel == 'panel2' else '지원내용'
        try:
            df.at[idx, key] = wait.until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, sel))
            ).text.strip()
        except TimeoutException:
            df.at[idx, key] = ''
    elif panel == 'panel4':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel4 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '신청기간' in title:
                    df.at[idx, '신청기간'] = text
                elif '신청방법' in title:
                    df.at[idx, '신청방법'] = text
            except NoSuchElementException:
                continue
    elif panel == 'panel5':
        for wrap in driver.find_elements(By.CSS_SELECTOR, '#panel5 .detail-wrap'):
            try:
                title = wrap.find_element(By.CSS_SELECTOR, '.detail-title').text
                text = wrap.find_element(By.CSS_SELECTOR, '.detail-desc').text.strip()
                if '접수기관' in title:
                    df.at[idx, '접수기관'] = text
                elif '문의처' in title:
                    df.at[idx, '문의처'] = text
            except NoSuchElementException:
                continue

# ——————————————
# 4) 메인 루프: 모든 행 저장 및 체크포인트 기록
# ——————————————
for idx, row in df.iterrows():
    url = row.get('detail_url')
    if not url:
        continue
    # 한번이라도 처리된 URL 스킵
    if url in checkpoint:
        print(f"[{idx+1}] 스킵됨: {url}")
        continue
    driver.get(url)
    human_delay(1, 2)
    # panel1 스크래핑
    scrape_panel1(idx)
    # 패널 순서대로 스크래핑
    for panel in ['panel3', 'panel2', 'panel4', 'panel5']:
        scrape_tab(idx, panel)
    # 누락 항목 기본값 처리
    missing = [c for c in cols if not df.at[idx, c].strip()]
    for c in missing:
        df.at[idx, c] = ''  # 빈 문자열 그대로 유지
    # CSV 저장
    record = row.to_dict()
    for c in cols:
        record[c] = df.at[idx, c]
    pd.DataFrame([record]).to_csv(
        OUTPUT_CSV_PATH, mode='a', index=False, header=False, encoding='utf-8-sig'
    )
    # 체크포인트 저장 (누락 여부 관계 없이)
    checkpoint[url] = True
    with open(CHECKPOINT_PATH, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)
    print(f"[{idx+1}/{len(df)}] 저장완료: {url}, 누락={missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')


[1/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/315000000114?administOrgCd=ALL, 누락=[]
[2/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/B27000100043?administOrgCd=ALL, 누락=[]
[3/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/420000000161?administOrgCd=ALL, 누락=[]
[4/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/537000000110?administOrgCd=ALL, 누락=[]
[5/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/319000000129?administOrgCd=ALL, 누락=[]
[6/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/319000000111?administOrgCd=ALL, 누락=[]
[7/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/313000000103?administOrgCd=ALL, 누락=[]
[8/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/303000000307?administOrgCd=ALL, 누락=['접수기관']
[9/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/534000000136?administOrgCd=ALL, 누락=[]
[10/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/374000000115?administOrgCd=ALL, 누락=[]
[11/22] 저장완료: https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3900000